<a href="https://colab.research.google.com/github/SarahSouzaPontes/LH_CD_SARAH_SOUZA_PONTES_regressao/blob/main/LH_CD_SARAH_SOUZA_PONTES_Somente_Predicao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#conexão com o google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

# Lendo os dados(todos)
# df_train = pd.read_excel("/content/drive/MyDrive/train.xlsx")
# df_test = pd.read_excel("/content/drive/MyDrive/test.xlsx")

# Lendo os dados(apenas 10000 linhas)

df_train = pd.read_excel("/content/drive/MyDrive/train.xlsx", nrows=10000)
df_test = pd.read_excel("/content/drive/MyDrive/test.xlsx", nrows=10000)

In [3]:
# convertendo o preco de centavos para reais (não entendi na verdade em que escala esta esse preco)
df_train['preco'] = df_train['preco']/100000000000

# TRATAMENTO DOS NULOS (MISSING VALUES)

In [4]:
#coluna veiculo_alienado 100 porcento de valores nulos,
# acima de 50 porcento melhor excluir a coluna do que tentar preencher(acho)
df_train.isnull().mean()

id                         0.0000
num_fotos                  0.0057
marca                      0.0000
modelo                     0.0000
versao                     0.0000
ano_de_fabricacao          0.0000
ano_modelo                 0.0000
hodometro                  0.0000
cambio                     0.0000
num_portas                 0.0000
tipo                       0.0000
blindado                   0.0000
cor                        0.0000
tipo_vendedor              0.0000
cidade_vendedor            0.0000
estado_vendedor            0.0000
anunciante                 0.0000
entrega_delivery           0.0000
troca                      0.0000
elegivel_revisao           0.0000
dono_aceita_troca          0.2557
veiculo_único_dono         0.6421
revisoes_concessionaria    0.6932
ipva_pago                  0.3365
veiculo_licenciado         0.4672
garantia_de_fábrica        0.8559
revisoes_dentro_agenda     0.8019
veiculo_alienado           1.0000
preco                      0.0000
dtype: float64

In [5]:
# Coluna id trás apenas informação de identificação, vamos excluir também
colunas_para_excluir = ['id', 'garantia_de_fábrica', 'revisoes_dentro_agenda',
                        'veiculo_alienado', 'veiculo_único_dono', 'revisoes_concessionaria']

df_train.drop(colunas_para_excluir, axis = 1, inplace=True)
df_test.drop(colunas_para_excluir, axis = 1, inplace=True)

In [6]:
# COLUNAS COM MENOS DE 50% NULAS VAMOS PREENCHER COM A MÉDIA SE FOR NUMERICA E A MODA SE FOR CATEGORICA
df_train.isnull().mean()

num_fotos             0.0057
marca                 0.0000
modelo                0.0000
versao                0.0000
ano_de_fabricacao     0.0000
ano_modelo            0.0000
hodometro             0.0000
cambio                0.0000
num_portas            0.0000
tipo                  0.0000
blindado              0.0000
cor                   0.0000
tipo_vendedor         0.0000
cidade_vendedor       0.0000
estado_vendedor       0.0000
anunciante            0.0000
entrega_delivery      0.0000
troca                 0.0000
elegivel_revisao      0.0000
dono_aceita_troca     0.2557
ipva_pago             0.3365
veiculo_licenciado    0.4672
preco                 0.0000
dtype: float64

In [7]:
from sklearn.impute import SimpleImputer


# preenchendo variaveis numericas nulas com a media
colunas_numericas_nulas = ['num_fotos']

imputer_numericas = SimpleImputer(strategy='mean')
df_train[colunas_numericas_nulas] = imputer_numericas.fit_transform(df_train[colunas_numericas_nulas])
df_test[colunas_numericas_nulas] = imputer_numericas.transform(df_test[colunas_numericas_nulas])

# preenchendo variaveis categoricas nulas com a moda
colunas_categoricas_nulas = ['dono_aceita_troca', 'ipva_pago', 'veiculo_licenciado']

imputer_categoricas = SimpleImputer(strategy='most_frequent')
df_train[colunas_categoricas_nulas] = imputer_categoricas.fit_transform(df_train[colunas_categoricas_nulas])
df_test[colunas_categoricas_nulas] = imputer_categoricas.transform(df_test[colunas_categoricas_nulas])

In [8]:
# não há mais valores nulos no dataframe
df_train.isnull().mean()

num_fotos             0.0
marca                 0.0
modelo                0.0
versao                0.0
ano_de_fabricacao     0.0
ano_modelo            0.0
hodometro             0.0
cambio                0.0
num_portas            0.0
tipo                  0.0
blindado              0.0
cor                   0.0
tipo_vendedor         0.0
cidade_vendedor       0.0
estado_vendedor       0.0
anunciante            0.0
entrega_delivery      0.0
troca                 0.0
elegivel_revisao      0.0
dono_aceita_troca     0.0
ipva_pago             0.0
veiculo_licenciado    0.0
preco                 0.0
dtype: float64

# Tratando as CATEGÓRICAS

In [9]:
# essas sao as variaveis numericas, quais delas devem ser tratadas como categoricas(fazer o dummy ou onehotencoder)?
df_train.describe()

,num_fotos,ano_de_fabricacao,ano_modelo,hodometro,num_portas,preco
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,10.395253,2016.758300,2017.806200,58517.725700,3.940700,99828.678553
std,3.506408,4.058045,2.677454,32830.633767,0.338223,95269.342339
min,8.000000,1985.000000,2006.000000,100.000000,2.000000,9.576975
25%,8.000000,2015.000000,2016.000000,31120.000000,4.000000,32073.205350
50%,8.000000,2018.000000,2018.000000,57380.500000,4.000000,82887.977985
75%,14.000000,2019.000000,2020.000000,82048.500000,4.000000,133473.661523
max,20.000000,2022.000000,2023.000000,379155.000000,4.000000,654991.187114


In [10]:
# Levando em conta que o número total de linhas do dataframe é 29584:

# utilizando a funçao unique().size verificamos o número de valores distintos.

# Coluna num_portas vamos dummizar porque gerará apenas 3 novas colunas(features)

# Coluna  preco e hodometro não precisa tratar pois são numéricas

# coluna ano_de_fabricacao, e ano_modelo,
# vamos codificar usando o label transformando os valores em categorias, 0,1,2 ...

# Coluna num_fotos não precisa mexer porque já esta como ordinal

print('num_fotos: ', df_train['num_fotos'].unique().size)
print('ano_de_fabricacao: ', df_train['ano_de_fabricacao'].unique().size)
print('ano_modelo: ', df_train['ano_modelo'].unique().size)
print('hodometro: ', df_train['hodometro'].unique().size)
print('num_portas: ', df_train['num_portas'].unique().size)
print('preco: ', df_train['preco'].unique().size)

# Definindo colunas numericas menos num_portas que será tratada como categorica :
colunas_numericas = ['num_fotos','ano_de_fabricacao','ano_modelo','hodometro','preco']

colunas_numericas_para_dummizar = ['num_portas']
colunas_numericas_para_codificar = ['ano_de_fabricacao', 'ano_modelo']

num_fotos:  13
ano_de_fabricacao:  34
ano_modelo:  15
hodometro:  9567
num_portas:  3
preco:  10000


In [11]:
# Definind colunas categoricas como sendo todas as colunas do df menos as numericas
colunas_categoricas = list(set(df_train.columns) - set(colunas_numericas))
colunas_categoricas

['ipva_pago',
 'modelo',
 'marca',
 'cidade_vendedor',
 'elegivel_revisao',
 'cor',
 'cambio',
 'anunciante',
 'num_portas',
 'troca',
 'estado_vendedor',
 'tipo_vendedor',
 'versao',
 'blindado',
 'tipo',
 'entrega_delivery',
 'dono_aceita_troca',
 'veiculo_licenciado']

In [12]:
# mesma tecnica do unique().size agora nas categoricas usando o for pra simplificar o codigo
for coluna in colunas_categoricas:
    print(coluna,': ', df_train[coluna].unique().size)

# verificamos as variaveis com muitos valores distintos e que não vale a pena dummificar(serao codificadas com label()):
colunas_categoricas_codificar = ['estado_vendedor', 'marca', 'versao', 'cidade_vendedor', 'modelo']

ipva_pago :  1
modelo :  391
marca :  38
cidade_vendedor :  343
elegivel_revisao :  1
cor :  6
cambio :  7
anunciante :  4
num_portas :  3
troca :  2
estado_vendedor :  24
tipo_vendedor :  2
versao :  1385
blindado :  2
tipo :  7
entrega_delivery :  2
dono_aceita_troca :  1
veiculo_licenciado :  1


In [13]:
# Definindo colunas categoricas para dummizar como sendo colunas_categoricas menos colunas_categoricas_codificar
colunas_categoricas_para_dummizar = list(set(colunas_categoricas) - set(colunas_categoricas_codificar))

# Definindo colunas totais para dummizar como sendo colunas_categoricas_para_dummizar mais colunas_numericas_para_dummizar
colunas_para_dummizar = list(colunas_categoricas_para_dummizar + colunas_numericas_para_dummizar)
colunas_para_dummizar

['ipva_pago',
 'elegivel_revisao',
 'cor',
 'cambio',
 'anunciante',
 'num_portas',
 'troca',
 'tipo_vendedor',
 'blindado',
 'tipo',
 'entrega_delivery',
 'dono_aceita_troca',
 'veiculo_licenciado',
 'num_portas']

In [14]:
# reunindo todas as colunas a codificar em uma mesma lista
colunas_para_codificar = list(colunas_categoricas_codificar + colunas_numericas_para_codificar)
colunas_para_codificar

['estado_vendedor',
 'marca',
 'versao',
 'cidade_vendedor',
 'modelo',
 'ano_de_fabricacao',
 'ano_modelo']

In [15]:
# dummizando as colunas_para_dummizar

# Veja que como dummizamos com criterio terminamos com um dataframe com apenas 39 colunas
# e não milhares de colunas como quando dummizamos todas as colunas indiscriminadamente

# df_train = pd.get_dummies(df_train, columns = colunas_para_dummizar)
# df_test = pd.get_dummies(df_test, columns = colunas_para_dummizar)

from sklearn.preprocessing import OneHotEncoder
meu_encoder = OneHotEncoder(drop='first',sparse_output=False, handle_unknown='ignore')

df_train = df_train.join(pd.DataFrame(meu_encoder.fit_transform(df_train[colunas_para_dummizar]), columns=meu_encoder.get_feature_names_out()))
df_train.drop(colunas_para_dummizar, axis = 1, inplace = True)

df_test = df_test.join(pd.DataFrame(meu_encoder.transform(df_test[colunas_para_dummizar]), columns=meu_encoder.get_feature_names_out()))
df_test.drop(colunas_para_dummizar, axis = 1, inplace = True)

df_train

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [2, 9] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


,num_fotos,marca,modelo,versao,ano_de_fabricacao,ano_modelo,hodometro,cidade_vendedor,estado_vendedor,preco,...,blindado_S,tipo_Hatchback,tipo_Minivan,tipo_Perua/SW,tipo_Picape,tipo_Sedã,tipo_Utilitário esportivo,entrega_delivery_True,num_portas_3,num_portas_4
0,8.0,NISSAN,KICKS,1.6 16V FLEXSTART SL 4P XTRONIC,2017,2017.0,67772.0,Rio de Janeiro,São Paulo (SP),74732.590084,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,8.0,JEEP,COMPASS,2.0 16V FLEX LIMITED AUTOMÁTICO,2017,2017.0,62979.0,Belo Horizonte,Minas Gerais (MG),81965.332634,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,16.0,KIA,SORENTO,2.4 16V GASOLINA EX 7L AWD AUTOMÁTICO,2018,2019.0,44070.0,Santos,São Paulo (SP),162.824814,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
3,14.0,VOLKSWAGEN,AMAROK,2.0 HIGHLINE 4X4 CD 16V TURBO INTERCOOLER DIES...,2013,2015.0,85357.0,Sorocaba,São Paulo (SP),123681.358857,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
4,8.0,SSANGYONG,KORANDO,2.0 GLS 4X4 16V TURBO DIESEL 4P AUTOMÁTICO,2013,2015.0,71491.0,Rio de Janeiro,Rio de Janeiro (RJ),8241.976389,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,16.0,VOLKSWAGEN,VIRTUS,1.0 200 TSI COMFORTLINE AUTOMÁTICO,2021,2021.0,22892.0,São Paulo,São Paulo (SP),56742.947239,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
9996,15.0,FIAT,TORO,1.8 16V EVO FLEX FREEDOM AT6,2018,2018.0,60388.0,São Paulo,São Paulo (SP),97148.757483,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
9997,8.0,MERCEDES-BENZ,C 180,1.8 CGI COUPÉ TURBO 16V GASOLINA 2P AUTOMÁTICO,2013,2013.0,98910.0,São Paulo,São Paulo (SP),105666.065159,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9998,8.0,HYUNDAI,SANTA FÉ,2.7 MPFI GLS 7 LUGARES V6 24V GASOLINA 4P AUTO...,2006,2012.0,110777.0,São Paulo,São Paulo (SP),43754.355542,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [16]:
# A coluna 'modelo' estava dando este erro:
# TypeError: Encoders require their input to be uniformly strings or numbers. Got ['int', 'str'] label
# Como ela mixa valores numericos e string(object) precisei transformar ela toda em string
# pra depois poder codificar usando o label()

df_train['modelo'] = df_train['modelo'].astype("string")
df_test['modelo'] = df_test['modelo'].astype("string")

In [17]:
# codificando as colunas_para_codificar
from sklearn.preprocessing import LabelEncoder

for coluna in colunas_para_codificar:
    label_encoder = LabelEncoder()
    df_train[coluna] = label_encoder.fit_transform(df_train[coluna])
    df_test[coluna] = label_encoder.fit_transform(df_test[coluna])

print(df_train[colunas_para_codificar])

      estado_vendedor  marca  versao  cidade_vendedor  modelo  \
0                  22     25     327              253     221   
1                  10     16     638               29     115   
2                  22     17    1027              273     320   
3                  22     36     790              279      52   
4                  18     31     782              253     223   
...               ...    ...     ...              ...     ...   
9995               22     36      16              297     363   
9996               22      9     537              297     342   
9997               22     22     564              297      70   
9998               22     12    1094              297     311   
9999               22     12      30              198     202   

      ano_de_fabricacao  ano_modelo  
0                    28           8  
1                    28           8  
2                    29          10  
3                    24           6  
4                    24      

In [18]:
df_train

,num_fotos,marca,modelo,versao,ano_de_fabricacao,ano_modelo,hodometro,cidade_vendedor,estado_vendedor,preco,...,blindado_S,tipo_Hatchback,tipo_Minivan,tipo_Perua/SW,tipo_Picape,tipo_Sedã,tipo_Utilitário esportivo,entrega_delivery_True,num_portas_3,num_portas_4
0,8.0,25,221,327,28,8,67772.0,253,22,74732.590084,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,8.0,16,115,638,28,8,62979.0,29,10,81965.332634,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,16.0,17,320,1027,29,10,44070.0,273,22,162.824814,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
3,14.0,36,52,790,24,6,85357.0,279,22,123681.358857,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
4,8.0,31,223,782,24,6,71491.0,253,18,8241.976389,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,16.0,36,363,16,32,12,22892.0,297,22,56742.947239,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
9996,15.0,9,342,537,29,9,60388.0,297,22,97148.757483,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
9997,8.0,22,70,564,24,4,98910.0,297,22,105666.065159,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9998,8.0,12,311,1094,17,3,110777.0,297,22,43754.355542,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


# Realizando a predicao no dataframe de validacao para verificar o score

In [19]:
# Esquecemos de excluir a coluna alvo 'preco' do df_train
x = df_train.drop('preco', axis = 1)
y = df_train['preco']

# Split the data into df_training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)

# Optionally, scale the data using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and df_train the Linear Regression model
linear_regression = LinearRegression()
linear_regression.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = linear_regression.predict(X_test_scaled)
y_pred

array([ 94293.14679732, 123502.13231168,  78259.5688662 , ...,
       125985.51780674, 104733.62918867, 102943.88600953])

In [20]:
# Avaliando o modelo
mae = metrics.mean_absolute_error(y_test, y_pred)
mse = metrics.mean_squared_error(y_test, y_pred)
r2 = metrics.r2_score(y_test, y_pred)

print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("R-squared:", r2)
# R-squared: 0.00339870331106662 # deveria estar entre 0.3 e 0.7

Mean Absolute Error: 67177.88947850722
Mean Squared Error: 10724930945.638672
R-squared: -0.00629200089131321


# Realizando a predicao em dados novos

In [21]:
# normalizando tambem o dataset de teste
df_test_scaled = scaler.transform(df_test)

# realizando a predicao nos dados novos do dataset de teste
y_pred_novos = linear_regression.predict(df_test_scaled)
y_pred_novos

array([ 97400.93620131,  89261.95963298, 123350.0741982 , ...,
        99680.76219061,  84767.84317755,  98062.97991671])

In [22]:
# imprimindo os pesos de cada coluna na predicao
print(linear_regression.coef_)

[ 5969.32128809 -2289.72324674 -1024.66736371  2893.93019908
  -743.27980562 -1280.73260801 -3595.52490075 -2788.13046705
  1060.56106796  -279.57230194  1319.18268392 -1631.17031849
  -487.16296255  5271.97347097 -1236.04068121 -9605.88033994
  -424.67421472 -5570.7817403  -8000.08529301   469.88423931
  -471.77855583   236.85257395  6238.0969377    232.61279929
   255.54781054  -628.56968894 -3139.48562137   731.87448393
 -1927.45747944  -206.47453458   524.76774145 -1756.21979797
 -3422.0176966   3221.01614572  2460.59984626   232.61279929
   255.54781054]
